# Part 1: Network Creation

In [6]:
import requests
import json
import re
import networkx as nx
import numpy as np
import os
from io import StringIO
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
from collections import Counter
import nltk
import pandas as pd

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/joaomata/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1. Fetch list of countries

In [24]:
baseurl = "https://en.wikipedia.org/w/api.php?"
action = "action=query"
title = "titles=Member_states_of_the_United_Nations"
content = "prop=revisions&rvprop=content"
dataformat ="format=json"

query = "{}{}&{}&{}&{}".format(baseurl, action, content, title, dataformat)
print(query)

https://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&titles=Member_states_of_the_United_Nations&format=json


Fetch raw data

In [25]:
headers = {"User-Agent" : "MyWikipediaClient/1.0 (example@example.com)"} # just use this dict as-is.
wikiresponse = requests.get(query, headers=headers)
wikidata = wikiresponse.text
dict_data=json.loads(wikidata)

In [26]:
raw_data=dict_data['query']['pages']['31969']['revisions'][0]['*']
raw_data

'{{Short description|none}}\n{{For|the League of Nations|Member states of the League of Nations}}\n{{Featured list}}\n{{pp-pc|small=yes}}\n{{Use dmy dates|date=February 2025}}\n{{Use Oxford spelling|date = February 2025}}\n\n[[File:United Nations (Member States and Territories).svg|thumb|\n{{legend|#009edb|193 United Nations member states}}\n{{legend|#9edb00|2 [[United Nations General Assembly observers#Current non-member observers|UN General Assembly observer states]] (the [[Holy See]]{{efn|The [[sovereignty|sovereign entity]] with [[state (polity)|statehood]] over the territory of the [[Vatican City|Vatican City State]].}} and [[Palestine]])}}\n{{legend|#db9e00|2 non-member states (the [[Cook Islands]] and [[Niue]])<ref>{{Cite web|url = https://www.un.org/en/development/desa/population/publications/pdf/policy/WPP2013/Chapters/a_Preface.pdf |title = UN: "Data for 197 countries: all 193 member states, two observer states (the Holy See and Palestine), and two non-member states (Cook Isl

Get countries using flag because they all appear as one of these: {flag|country}, {flagdeco|country}, {flagcountry|country}

This list of countries is a list of strings that have:
- capital letters
- there are white spaces (substituir later por underscore)

This list doesnt work to access wikipedia.

In [27]:
countries = re.findall(r"\{flag(?:deco|country)?\|([^|}]+)", raw_data)
no_countries = len(countries)
print("Number of countries:", no_countries)

Number of countries: 193


## 2. Fetch wikipages of countries

Countries doesnt work in wikipedia

Countries_clean works in wikipedia

In [28]:
countries = [re.sub(r"\s*\(.*?\)", "", c).strip() for c in countries]
countries_clean=[]
for c in countries:
    if c=="Bahamas":
        c="The_Bahamas"
    if c=="Cabo Verde":
        c="Cape_Verde"
    if c=="Côte d'Ivoire":
        c="Ivory_Coast"
    if c=="Czechia":
        c="Czech_Republic"
    if c=="Guinea Bissau":
        c="Guinea-Bissau"
    if c=="Lao People's Democratic Republic":
        c="Laos"
    if c=="Republic of Korea":
        c="South_Korea"
    if c=="Republic of Moldova":
        c="Moldova"
    if c=="Russian Federation":
        c="Russia"
    if c=="Syrian Arab Republic":
        c="Syria"
    if c=="Sao Tome and Principe":
        c="São_Tomé_and_Príncipe"
    if c=="United Kingdom of Great Britain and Northern Ireland":
        c="United_Kingdom"
    if c=="United Republic of Tanzania":
        c="Tanzania"
    if c=="Türkiye":
        c="Turkey"
    if c=="United States of America":
        c="United_States"
    if c=="Congo":
        c="Republic_of_the_Congo"
    new_c = c.replace(" ", "_")
    countries_clean.append(new_c)

In [29]:
len(countries_clean) == len(countries)

True

Vai buscar a secção History de cada pais, acedendo a wikipedia, mas nao tem o texto das referencias no meio

In [30]:
# baseurl = "https://en.wikipedia.org/w/api.php?"
# headers = {"User-Agent" : "MyWikipediaClient/1.0 (example@example.com)"} # just use this dict as-is.

# folder_name = "countries_pages"
# os.makedirs(folder_name, exist_ok=True)

# for c in countries_clean:

#     print(c)

#     #get index for history section
#     params_sections = {"action": "parse", "page": c, "prop": "sections", "format": "json"}
#     r = requests.get(baseurl, params=params_sections, headers=headers)
#     sections_data = r.json()
#     for s in sections_data["parse"]["sections"]:
#     #print(s["index"], s["line"])
#        if s["line"] == "History":
#         history_index = s["index"]
#         break

#     #get text of history section
#     params_text = {"action": "parse", "page": c, "prop": "wikitext", "section": history_index, "format": "json"}
#     r2 = requests.get(baseurl, params=params_text, headers=headers)
#     r2_text = r2.text
#     history_data=json.loads(r2_text)
#     history_text=history_data["parse"]["wikitext"]["*"]
#     history_text = re.sub(r"<ref\b[^>]*/>", "", history_text)
#     clean_text = re.sub(r"<ref\b[^>]*>.*?</ref>", "", history_text, flags=re.DOTALL)
    
#     file_path = os.path.join(folder_name, c+".txt")
#     with open(file_path, "w", encoding="utf-8") as f:
#         f.write(clean_text)

## 3. Create network

In [3]:
folder_name = "countries_pages"
all_items = os.listdir(folder_name)
len(all_items) == no_countries

NameError: name 'no_countries' is not defined

Criar grafo sem atributos, encontra todos os paises, nao so os que tem link.

In [4]:
G = nx.DiGraph()

for filename in all_items:

    #ir buscar texto deste file
    country_file_path = os.path.join(folder_name, filename)
    with open(country_file_path, "r", encoding="utf-8") as f:
        country_text = f.read()

    clean_text = re.sub(r"<ref.*?>.*?</ref>", "", country_text, flags=re.DOTALL)

    #get name of node, filename é str com .txt
    country=filename.split('.')[0]
    G.add_node(country)

    mentioned_countries = set()
    for other_country in countries_clean:
        new_other = other_country.replace("_", " ")
        if other_country == country:
            continue  # skip self
        # search anywhere in text, case-insensitive, word boundaries
        if re.search(rf"\b{re.escape(new_other)}\b", clean_text, flags=re.IGNORECASE) and other_country!=country and other_country not in mentioned_countries:
            mentioned_countries.add(other_country)
            G.add_edge(country, other_country)

NameError: name 'countries_clean' is not defined

In [33]:
print(G.number_of_nodes())
print(G.number_of_edges())

193
2119


In [34]:
source = "Denmark"  # the node you want to inspect

# returns an iterator of nodes that the country points to
outgoing_nodes = list(G.successors(source))

print(f"Nodes that {source} points to:")
for node in outgoing_nodes:
    print(" -", node)

Nodes that Denmark points to:
 - Canada
 - Estonia
 - France
 - Germany
 - Ghana
 - Iceland
 - India
 - Ireland
 - Norway
 - Russia
 - Sri_Lanka
 - Sweden
 - United_Kingdom


In [35]:
# Remove Moldova as its history section is too short
G.remove_node("Moldova")
print(G.number_of_nodes())
print(G.number_of_edges())

192
2115


Guardar grafo sem atributos para usar na sentiment analysis

In [36]:
# Save the enriched graph to a GraphML file
nx.write_graphml(G, "world_graph.graphml")
print("Graph saved to 'world_graph.graphml'")

Graph saved to 'world_graph.graphml'


## 4. Calcular sentiment score

In [7]:
# load graph
G = nx.read_graphml("data/world_graph.graphml")

In [8]:
scores_df = pd.read_csv("data/country_scores.csv")

for row in scores_df.itertuples(index=False):
    
    country_name = row.Country

    if country_name in G.nodes:
        G.nodes[country_name]["sentiment_score"] = row.Overall_Score  
    else:
        print(f"Skipping {country_name} (mapped to '{country_name}') as it is not in the graph.")

print("\nFinished adding attributes to graph nodes.")


Finished adding attributes to graph nodes.


In [9]:
# Check the attributes of a sample node
sample_node = "São_Tomé_and_Príncipe"
print(f"Attributes for {sample_node}:")
print(G.nodes[sample_node])

Attributes for São_Tomé_and_Príncipe:
{'sentiment_score': 5.0}


## 4. Add attributes from dataset

In [10]:
# Load the csv file
df = pd.read_csv("data/world-data-2023.csv")

# 1. .str.lower() -> Make all lowercase
# 2. .str.replace(r'[^a-z0-9]+', '_', regex=True) -> Replace any non-alphanumeric characters with '_'
# 3. .str.strip('_') -> Remove any leading/trailing '_'
df.columns = (df.columns.str.lower()
                .str.replace(r'[^a-z0-9]+', '_', regex=True)
                .str.strip('_'))

print("Cleaned column names:", df.columns.tolist())

# Dictionary to map CSV names to Graph node names (they are different for some countries)
COUNTRY_MAP = {
    "Federated_States_of_Micronesia": "Micronesia",
    "East_Timor": "Timor-Leste",
    "Republic_of_Ireland": "Ireland",
    "Sao_Tome_and_Principe": "São_Tomé_and_Príncipe"}

# These ecountries are not in the graph and should be skipped in the matching process
COUNTRIES_TO_SKIP = {
    "Vatican_City",
    "Palestinian_National_Authority",
    "Moldova"}

# extra data (continent mapping)
CONTINENTS = {
    "Africa": [
        "Algeria","Angola","Benin","Botswana","Burkina_Faso","Burundi",
        "Cameroon","Cape_Verde","Central_African_Republic","Chad","Comoros",
        "Congo","Democratic_Republic_of_the_Congo","Djibouti","Egypt",
        "Equatorial_Guinea","Eritrea","Eswatini","Ethiopia","Gabon",
        "Gambia","Ghana","Guinea","Guinea-Bissau","Ivory_Coast","Kenya",
        "Lesotho","Liberia","Libya","Madagascar","Malawi","Mali",
        "Mauritania","Mauritius","Morocco","Mozambique","Namibia","Niger",
        "Nigeria","Republic_of_the_Congo","Rwanda","São_Tomé_and_Príncipe",
        "Senegal","Seychelles","Sierra_Leone","Somalia","South_Africa",
        "South_Sudan","Sudan","Tanzania","The_Gambia","Togo","Tunisia",
        "Uganda","Zambia","Zimbabwe"
    ],

    "Asia": [
        "Afghanistan","Armenia","Azerbaijan","Bahrain","Bangladesh",
        "Bhutan","Brunei","Cambodia","China","Cyprus","Georgia","India",
        "Indonesia","Iran","Iraq","Israel","Japan","Jordan","Kazakhstan",
        "Kuwait","Kyrgyzstan","Laos","Lebanon","Malaysia","Maldives",
        "Mongolia","Myanmar","Nepal","North_Korea","Oman","Pakistan",
        "Philippines","Qatar","Saudi_Arabia","Singapore","South_Korea",
        "Sri_Lanka","Syria","Tajikistan","Thailand","Timor-Leste",
        "Turkey","Turkmenistan","United_Arab_Emirates","Uzbekistan",
        "Vietnam","Yemen"
    ],

    "Europe": [
        "Albania","Andorra","Austria","Belarus","Belgium","Bosnia_and_Herzegovina",
        "Bulgaria","Croatia","Czech_Republic","Denmark","Estonia","Finland",
        "France","Germany","Greece","Hungary","Iceland","Ireland","Italy",
        "Kazakhstan","Kosovo","Latvia","Liechtenstein","Lithuania","Luxembourg",
        "Malta","Moldova","Monaco","Montenegro","Netherlands","North_Macedonia",
        "Norway","Poland","Portugal","Romania","Russia","San_Marino","Serbia",
        "Slovakia","Slovenia","Spain","Sweden","Switzerland","Ukraine",
        "United_Kingdom"
    ],

    "North_America": [
        "Antigua_and_Barbuda","Bahamas","Barbados","Belize","Canada","Costa_Rica",
        "Cuba","Dominica","Dominican_Republic","El_Salvador","Grenada","Guatemala",
        "Haiti","Honduras","Jamaica","Mexico","Nicaragua","Panama",
        "Saint_Kitts_and_Nevis","Saint_Lucia","Saint_Vincent_and_the_Grenadines",
        "The_Bahamas","Trinidad_and_Tobago","United_States"
    ],

    "South_America": [
        "Argentina","Bolivia","Brazil","Chile","Colombia","Ecuador","Guyana",
        "Paraguay","Peru","Suriname","Uruguay","Venezuela"
    ],

    "Oceania": [
        "Australia","Fiji","Kiribati","Marshall_Islands","Micronesia","Nauru",
        "New_Zealand","Palau","Papua_New_Guinea","Samoa","Solomon_Islands",
        "Tonga","Tuvalu","Vanuatu"
    ]
}


for row in df.itertuples(index=False):
    
    # Get the country name and clean it (e.g., "United States" -> "United_States")
    country_name = row.country.replace(" ", "_")

    # 1. Check if this country should be skipped
    if country_name in COUNTRIES_TO_SKIP:
        print(f"Skipping {country_name} (NOT IN GRAPH).")
        continue

    # 2. Check if the name needs mapping
    graph_country_name = COUNTRY_MAP.get(country_name, country_name)

    # 3. Add attributes if the name exists in the graph
    if graph_country_name in G.nodes:
        # Convert the row (a namedtuple) to a dictionary
        attributes = row._asdict()
        
        # Use .update() to add all data from the row as attributes to the node
        gdp = row.gdp
        gdp_clean = int(re.sub(r"[$,\s]", "", gdp))
        pop = row.population
        pop_clean = int(re.sub(r"[,\s]+", "", pop))
        gdp_capita = gdp_clean/pop_clean
        language = row.official_language
        
        # get continent from CONTINENTS
        continent = None
        for cont, countries in CONTINENTS.items():
            if graph_country_name in countries:
                continent = cont
                break
        G.nodes[graph_country_name]["continent"] = continent   
        G.nodes[graph_country_name]["gdp"] = round(gdp_capita, 3)
        G.nodes[graph_country_name]["official_language"] = language
        
        
    else:
        # This will catch names not in the graph, skip list, or map
        print(f"Skipping {country_name} (mapped to '{graph_country_name}') as it is not in the graph.")

print("\nFinished adding attributes to graph nodes.")


Cleaned column names: ['country', 'density_p_km2', 'abbreviation', 'agricultural_land', 'land_area_km2', 'armed_forces_size', 'birth_rate', 'calling_code', 'capital_major_city', 'co2_emissions', 'cpi', 'cpi_change', 'currency_code', 'fertility_rate', 'forested_area', 'gasoline_price', 'gdp', 'gross_primary_education_enrollment', 'gross_tertiary_education_enrollment', 'infant_mortality', 'largest_city', 'life_expectancy', 'maternal_mortality_ratio', 'minimum_wage', 'official_language', 'out_of_pocket_health_expenditure', 'physicians_per_thousand', 'population', 'population_labor_force_participation', 'tax_revenue', 'total_tax_rate', 'unemployment_rate', 'urban_population', 'latitude', 'longitude']
Skipping Vatican_City (NOT IN GRAPH).
Skipping Moldova (NOT IN GRAPH).
Skipping Palestinian_National_Authority (NOT IN GRAPH).

Finished adding attributes to graph nodes.


#### adding the continent as an attribute (manual from list)

In [11]:
# Check the attributes of a sample node
sample_node = "Portugal"
print(f"Attributes for {sample_node}:")
print(G.nodes[sample_node])

Attributes for Portugal:
{'sentiment_score': 6.0, 'continent': 'Europe', 'gdp': 23145.041, 'official_language': 'Portuguese'}


In [12]:
## Diagnostic Check (Optional)
problem_attributes = set()
for node, data in G.nodes(data=True):
    for k, v in data.items():
        if v is None:
            problem_attributes.add(k)
            print(f"Node '{node}' has None value for attribute '{k}'.")

if problem_attributes:
    print(f"The following attributes contain 'None' values: {problem_attributes}")
    
else:
    print("No None values found.")

No None values found.


In [13]:
G.number_of_nodes()

192

In [14]:
# Save the enriched graph to a GraphML file
nx.write_graphml(G, "data/world_graph_attributes.graphml")
print("Graph saved to 'data/world_graph_attributes.graphml'")

Graph saved to 'data/world_graph_attributes.graphml'


In [16]:
# chek if all nodes have sentiment_score and gdp attributes
missing_sentiment = []
missing_gdp = []
for node, data in G.nodes(data=True):
    if "sentiment_score" not in data:
        missing_sentiment.append(node)
    if "gdp" not in data:
        missing_gdp.append(node)
    if "continent" not in data:
        print(f"Node '{node}' is missing 'continent' attribute.")
    
print("Nodes missing 'sentiment_score':", missing_sentiment)
print("Nodes missing 'gdp':", missing_gdp)


Nodes missing 'sentiment_score': []
Nodes missing 'gdp': []
